In [1]:
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')

# SLIDE (1) Основы метрик классификации

На вход подаются 2 массива $y_{real}$ - реальные значения бинарных классов и $y_{pred}$ - предсказанные значения бинарных классов. Вам необходимо посчитать, не используя стандартные функции, метрики: 
* $accuracy$
* $precision$
* $recall$
* $F_1$

Возвращать числа нужно именно в данном порядке.

### Sample 1
#### Input:
```python
y_real = np.array([0, 1, 0, 0, 1, 1, 1, 1])
y_pred = np.array([0, 1, 1, 0, 1, 1, 0, 0])
```
#### Output:
```python
0.625, 0.75, 0.6, 0.66
```

In [3]:
import numpy as np

def main_metrics(y_real, y_pred) -> (float, float, float, float):
    tp, fn, fp, tn = 0, 0, 0, 0
    for i in range(y_real.shape[0]):
        if y_pred[i] == 1:
            if y_real[i] == 1:
                tp += 1
            else:
                fp +=1
        else:
            if y_real[i] == 1:
                fn += 1
            else:
                tn += 1
    acc = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    return acc, precision, recall, f1

y_real = np.array([0, 1, 0, 0, 1, 1, 1, 1])
y_pred = np.array([0, 1, 1, 0, 1, 1, 0, 0])
print(main_metrics(y_real, y_pred))

(0.625, 0.75, 0.6, 0.6666666666666665)


# SLIDE (1) Основы метрик регрессии

Решаем задачу регрессии. На вход подаются 2 массива $y_{real}$ - реальные значения функции и $y_{pred}$ - предсказанные значения функции. Вам необходимо посчитать, не используя стандартные функции, метрики: 
* $R^2score$
* $MAE$ - `mean_absolute_error`
* $MSE$ - `mean_squared_error`
* $MSLE$ - `mean_squared_log_error`

Возвращать числа нужно именно в данном порядке.

Формулы для метрик - ищите в интернете, это часть задания. Можете сверяться с реальными метриками в `sklearn.metrics`.

Все числа в тестах больше 0, поэтому $MSLE$ будет считаться корректно
### Sample 1
#### Input:
```python
y_real = np.array([1, 2, 3, 4, 6])
y_pred = np.array([1, 3, 2, 4, 5])
```
#### Output:
```python
0.797297, 0.6, 0.6, 0.037856
```

In [22]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error
import numpy as np
from math import log

def reg_metrics(y_real, y_pred) -> (float, float, float, float):
    def my_mae(y_real, y_pred):
        n = y_real.shape[0]
        metric = 0
        for i in range(n):
            metric += abs(y_real[i] - y_pred[i])
        return metric / n

    def my_mse(y_real, y_pred):
        n = y_real.shape[0]
        metric = 0
        for i in range(n):
            metric += pow(y_real[i] - y_pred[i], 2)
        return metric / n

    def my_msle(y_real, y_pred):
        n = y_real.shape[0]
        metric = 0
        for i in range(n):
            metric += pow(log(y_real[i] + 1) - log(y_pred[i] + 1), 2)
        return metric / n

    def my_r2(y_real, y_pred):
        n = y_real.shape[0]
        res = 0
        tot = 0
        for i in range(n):
            res += pow(y_real[i] - y_pred[i], 2)
            tot += pow(y_real[i] - np.mean(y_real), 2)
        return 1 - res/tot
    
    mae = my_mae(y_real, y_pred)
    mse = my_mse(y_real, y_pred)
    msle = my_msle(y_real, y_pred)
    r2 = my_r2(y_real, y_pred)
    return r2, mae, mse, msle

y_real = np.array([1, 2, 3, 4, 6])
y_pred = np.array([1, 3, 2, 4, 5])
print(reg_metrics(y_real, y_pred))

(0.7972972972972974, 0.6, 0.6, 0.03785687634230184)


# SLIDE (1) LogLoss

Реализуйте свой LogLoss метрику.

В результат $y_{real}$ - передаются значения бинарных классов. В $y_{prob}$ - передаются вероятности $P(y_{pred}=1)$. Чтобы обработать краевые случаи в $y_{prob}$: 
* $0$ нужно заменить на `eps`
* $1$ нужно заменить на ($1$ - `eps`).
### Sample 1
#### Input:
```python
y_real = np.array([  1,   1,   0,   0,   1,   0,   1,   0])
y_prob = np.array([  1, 0.8, 0.2, 0.2, 0.4, 0.4, 0.6,   0])
```
#### Output:
``` python
    0.325921
```

In [13]:
from math import log

def logloss(y_real: np.array, y_prob: np.array, eps=1e-15) -> float:
    y_prob[y_prob == 1] = 1 - eps
    y_prob[y_prob == 0] = eps
    metric = 0
    n = y_real.shape[0]
    for i in range(y_real.shape[0]):
        metric += (y_real[i] * log(y_prob[i])) + (((1 - y_real[i]) * log(1 - y_prob[i])))
    return - metric / n

y_real = np.array([  1,   1,   0,   0,   1,   0,   1,   0])
y_pred = np.array([  1, 0.8, 0.2, 0.2, 0.4, 0.4, 0.6,   0])
print(logloss(y_real, y_pred))

0.3259215791685959


# SLIDE (2) Нахождение Roc-curve

Вам на вход даны $y_{real}$ и массив вероятностей $P(y_{pred}=1)$ необходимо реализовать функцию `roc-curve`, которая вернет 2 массива различных значений fpr и tpr, для дальнейшего построения Roc кривой.

Можно считать, что все вероятности ограничены $decimal=2$ (у каждого числа не более 2-х знаков после запятой)

### Sample
#### Input:
```python
y_real = np.array([  1,   1,   0,   0,   0,   1,   0,   1,   0])
y_prob = np.array([0.8, 0.8, 0.2, 0.2, 0.6, 0.4, 0.6, 0.6, 0.4])
```
#### Output:
```python
array([0.,  0.,  0.4, 0.6, 1. ]), #fpr
array([0., 0.5, 0.75,  1., 1. ])  #tpr
```

### Sample 2
#### Input:
```python
y_real = np.array([  1,   1,   0,   0,   1,   0,   1,   0])
y_prob = np.array([0.8, 0.8, 0.2, 0.2, 0.4, 0.4, 0.6, 0.6])
```
#### Output:
```python
array([0.,  0., 0.25, 0.5, 1. ]), #fpr
array([0., 0.5, 0.75,  1., 1. ])  #tpr

или 

array([0.,  0., 0.5, 1. ]), #fpr
array([0., 0.5,  1., 1. ])  #tpr
```

Обратите внимание на 2 пример: roc кривая, которая задается ими - одинаковая. Точка, которая уходит, находится на прямой между двумя соседними, в целом такие точки можно убирать, но будут приниматься оба варианта. Функция `sklearn.metrics.roc_curve` возвращает второй вариант.

In [45]:
from sklearn import metrics

def roc(y_real: np.array, y_prob: np.array) -> (np.array, np.array):
    thresholds = list(set(y_prob))
    thresholds.append(1)
    thresholds.sort(reverse=True)
    fprs = []
    tprs = []
    for threshold in thresholds:
        tp, fp, tn, fn = 0, 0, 0, 0
        for i in range(len(y_real)):
            value = y_real[i]
            prob = y_prob[i]
            if prob >= threshold:
                if value == 1:
                    tp += 1
                else:
                    fp += 1
            else:
                if value == 1:
                    fn += 1
                else:
                    tn += 1
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        fprs.append(fpr)
        tprs.append(tpr)
    return fprs, tprs

y_real = np.array([  1,   1,   0,   0,   0,   1,   0,   1,   0])
y_prob = np.array([0.8, 0.8, 0.2, 0.2, 0.6, 0.4, 0.6, 0.6, 0.4])
roc(y_real, y_prob)


tp  0  fp  0  tn  5  fn 4
tp  2  fp  0  tn  5  fn 2
tp  3  fp  2  tn  3  fn 1
tp  4  fp  3  tn  2  fn 0
tp  4  fp  5  tn  0  fn 0


([0.0, 0.0, 0.4, 0.6, 1.0], [0.0, 0.5, 0.75, 1.0, 1.0])

# SLIDE (1) Кросс-валидация

На вход падаются [данные](https://yadi.sk/d/6pUM_Ko-RtqiZg) и $k$ - число фолдов в кросс-валидации. Верните значения кросс-валидации по $k$ фолдам алгоритма `LogisticRegression` с метрикой `neg_log_loss`. 

В итоге должны получиться значения не ниже $-1$.

### Sample
#### Input:
```python
X = pd.read_csv('resources/train.csv').drop(columns=['y']).values
Y = pd.read_csv('resources/train.csv')['y']
k = 3
```
#### Output:
```python
array([-0.45456358, -0.41684932, -0.49260998])
```

In [48]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression as LR
import pandas as pd

def crossvalidate(X, y, k):
    retrun cross_val_score(LR(), X, y, cv=k)

X = pd.read_csv('resources/train.csv').drop(columns=['y']).values
Y = pd.read_csv('resources/train.csv')['y']
k = 3
crossvalidate(X, Y, k)

[0.75233645 0.79439252 0.78301887]


//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# SLIDE (1) GridSearch

C помощью [GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) найдите лучшие коэффициенты гиперпараметров `max_depth` и `min_samples_leaf` для классификатора [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) и верните обученный grid_search. 

* Пределы `max_depth` $(1, 10)$ 
* Пределы `min_samples_leaf` $(1, 10)$  
* Входные данные по [ссылке](https://yadi.sk/d/6pUM_Ko-RtqiZg)
* scoring - `precision`
* cv - $5$
* Другие параметры в `DecisionTreeClassifier` не указывать.

Не нужно Shuffl-ить данные, это может повлиять на ответ и в итоге задача не зачтется.

In [53]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier as DTC

def fit_gs(X: np.ndarray, y: np.ndarray) ->  GridSearchCV:
    params = { 'max_depth': np.arange(1, 10), 'min_samples_leaf': np.arange(1, 10) }
    gs = GridSearchCV(
        estimator = DTC(),
        param_grid = params,
        cv = 5,
        scoring = 'precision'
    )
    return gs.fit(X, y)

X = pd.read_csv('sonar_train.csv').drop(columns=['y']).values
Y = pd.read_csv('sonar_train.csv')['y']
fitted = fit_gs(X, Y)
fitted.cv_results_

{'mean_fit_time': array([0.00114317, 0.00173845, 0.00125279, 0.00158863, 0.00068178,
        0.00090456, 0.00076699, 0.00100813, 0.00071802, 0.00077796,
        0.00073571, 0.00071449, 0.00070229, 0.00071564, 0.00069013,
        0.00068402, 0.0008348 , 0.00086579, 0.00078945, 0.0007586 ,
        0.00075526, 0.00079341, 0.0007967 , 0.00075941, 0.00077863,
        0.0007627 , 0.00077581, 0.00114198, 0.00083623, 0.00082641,
        0.00085964, 0.00088696, 0.00086617, 0.00082865, 0.0008359 ,
        0.00084453, 0.00095582, 0.00089674, 0.00089478, 0.00087748,
        0.00090837, 0.00089521, 0.0008698 , 0.00088305, 0.00087485,
        0.00244317, 0.00099311, 0.0009635 , 0.00099597, 0.00114751,
        0.00113263, 0.00106044, 0.00104923, 0.00127478, 0.0012845 ,
        0.0011693 , 0.00119023, 0.00118089, 0.00123639, 0.00124702,
        0.00142064, 0.00136795, 0.00127606, 0.00158014, 0.00106664,
        0.00103641, 0.00098066, 0.00095778, 0.00096068, 0.00094299,
        0.00100689, 0.00092354,

# SLIDE (1) RandomSearch

C помощью [RandomSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) найдите лучшие коэффициенты гиперпараметров `max_depth` и `min_samples_leaf` для классификатора [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). 

Обучите $2$ `DTC`. Один с количеством итераций - $10$, другой с количеством итераций $50$. Верните обе обученные модели в соответствующем порядке.

* Пределы `max_depth` $(1, 10)$ 
* Пределы `min_samples_leaf` $(1, 10)$  
* Входные данные по [ссылке](https://yadi.sk/d/6pUM_Ko-RtqiZg)
* scoring - `precision`
* cv - $5$
* Другие параметры в `DecisionTreeClassifier` не указывать.

Не нужно Shuffl-ить данные, это может повлиять на ответ и в итоге задача не зачтется.

In [59]:
import numpy as np
from sklearn.model_selection import  RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier as DTC

def fit_gs(X: np.ndarray, y: np.ndarray) -> (RandomizedSearchCV, RandomizedSearchCV):
    params = { 'max_depth': np.arange(1, 10), 'min_samples_leaf': np.arange(1, 10) }
    rs10 = RandomizedSearchCV(
        estimator = DTC(),
        param_distributions = params,
        cv = 5,
        scoring = 'precision'
    )
    rs50 = RandomizedSearchCV(
        estimator = DTC(),
        param_distributions = params,
        cv = 5,
        scoring = 'precision',
        n_iter = 50
    )
    return rs10.fit(X, y), rs50.fit(X, y)

X = pd.read_csv('sonar_train.csv').drop(columns=['y']).values
Y = pd.read_csv('sonar_train.csv')['y']
rs10, rs50 = fit_gs(X, Y)
print(pd.DataFrame(rs10.cv_results_).sort_values(by='rank_test_score').head(3))
print(pd.DataFrame(rs50.cv_results_).sort_values(by='rank_test_score').head(3))

   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
1       0.001987      0.000261         0.007144        0.007229   
4       0.001024      0.000087         0.000926        0.000025   
8       0.001061      0.000189         0.002466        0.003015   

  param_min_samples_leaf param_max_depth  \
1                      7               4   
4                      6               8   
8                      8               5   

                                    params  split0_test_score  \
1  {'min_samples_leaf': 7, 'max_depth': 4}           0.840000   
4  {'min_samples_leaf': 6, 'max_depth': 8}           0.787879   
8  {'min_samples_leaf': 8, 'max_depth': 5}           0.781250   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
1           0.847826           0.766667           0.714286           0.741379   
4           0.793103           0.774194           0.741935           0.740741   
8           0.770492           0.769231           

# SLIDE (2) Моя Кросс-валидация

Вам необходимо реальзовать собственную функцию кросс-валидации. 

Напоминание, кросс-валидация: 
1. Разбивает, предварительно перемешав, данные на $k$ (заданное) `фолдов` данных (если нацело данные не делятся - хвост отбрасываем).
2. Берет один из фолдов за тестовую выборку, а все остальное за тренировачную.
3. Обучает эстиматор на тренировачных данныx 
4. Cчитает `score` обученной модели.
5. 2-4 шаги повторяются для всех $k$ фолдов 

Она должна принимать на вход: 
* `estimator`, который мы хотим обучить на разных фолдах данных
* $X, y$ - данные 
* `cv` ($k$) - объект разбиения `KFold` или количество фолдов, на которые мы бьем данные (по умолчанию 3)
* `scoring` - функция метрики, по которой оцениваем полученные результаты (замечание: в реальной `cross_val_score` здесь будет стоять объект `scorer`, у нас же будет стоять функция, например `metrics.accuracy_score`)

На выходе мы получаем массив длины $k$ из score значений.

Подсказка: используйте `model_selection.KFold` для разбиения выборки.

Подсказка: не нужно shuffl-ить данные при создании `KFold` из `cv`! 

### Sample
#### Input:
```python
estimator = LinearRegression()
X = np.array([[1],[2],[3],[4],[5]])
y = np.array([1, 2, 4, 4, 6])
cv = 2
scoring = sklearn.metrics.r2_score
```
#### Output:
```python
array([-2.64285714 -0.23611111])
```

In [78]:
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

def my_cross_val_score(estimator, X, y, cv=3, scoring=accuracy_score):
    if type(cv) == int:
        cv = KFold(n_splits=cv)
    scorings = []
    for train_index, test_index in cv.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        estimator.fit(X_train, y_train)
        y_pred = estimator.predict(X_test)
        scorings.append(scoring(y_test, y_pred))
    return scorings
    
estimator = LinearRegression()
X = np.array([[1],[2],[3],[4],[5]])
y = np.array([1, 2, 4, 4, 6])
cv = 2
scoring = r2_score
my_cross_val_score(estimator, X, y, cv, scoring)

TRAIN: [3 4] TEST: [0 1 2]
TRAIN: [0 1 2] TEST: [3 4]


[-2.6428571428571366, -0.2361111111111105]